In [ ]:
import os, sys

In [ ]:
base_dir = os.path.split(os.getcwd())[0]
os.chdir(os.path.join(base_dir))

In [ ]:
import pickle as pk

import numpy as np
import cv2
import sys
import argparse
import time
import copy
import datetime
import yaml

import matplotlib.pyplot as plt

from cranpose.utils import ARUCO_DICT, display_pose
from cranpose.estimators import PoseSingle

In [ ]:
k_matrix = "calib/calibration_matrix_hikvision.npy"
d_coeff = "calib/distortion_coefficients_hikvision.npy"
config_filename = "jupyters/marker_poses_warehouse.yaml"

edge_len = 0.7
marker_step = 10
n_markers = 100

type_ = "DICT_5X5_100"

if ARUCO_DICT.get(type_, None) is None:
    print(f"ArUCo tag type '{type_}' is not supported")
    sys.exit(0)

aruco_dict_type = ARUCO_DICT[type_]
calibration_matrix_path = k_matrix
distortion_coefficients_path = d_coeff

k = np.load(k_matrix)
d = np.load(d_coeff)


with open(config_filename, encoding='utf8') as f:
    marker_poses = yaml.load(f, Loader=yaml.FullLoader)


# with open(config_filename, encoding='utf8') as f:
#     marker_poses = yaml.load(f, Loader=yaml.FullLoader)

# Изменим калибровки так как разрешение потока отличается от разрешения калибровочного видео
k_video = copy.deepcopy(k)
k_video[0,0] = k_video[0,0] * 1280/2688
k_video[1,1] = k_video[1,1] * 720/1520

k_video[0,2] = k_video[0,2] * 1280/2688
k_video[1,2] = k_video[1,2] * 720/1520

In [ ]:
k_video

In [ ]:
estimator_front = PoseSingle(
    aruco_dict_type,
    -1,
    n_markers,
    marker_poses,
    edge_len,
    k_video,
    d,
    apply_kf = True,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = 1.25,
    # left_edge_weight_func  = f_left_x_04,
)

In [ ]:
# Test image preprocessing
image = "D:\\NKB\\cranepos_data\\frames4\\1000.png"

image = cv2.imread(image)
time.sleep(.1)

# sharpen with a kernel
# kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# image = cv2.filter2D(image, -1, kernel)

# sharpen with gaussian blur
# image_blurred = cv2.GaussianBlur(image,(15,15), cv2.BORDER_DEFAULT)
# image = cv2.addWeighted(image, 2.0, image_blurred, -1.0, 0)

# play with contrast-brightness
# contrast=0.5
# brightness = 50
# image = cv2.addWeighted(image, contrast, image, 0, brightness)

# gamma transformation
# gamma = 1.95
# image = np.array(255*(image / 255) ** gamma, dtype = 'uint8')

# crop and resize
# image = image[350:434, 500:650]


output = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize = (20,20))

plt.imshow(output)
plt.show()

In [ ]:
# Test image preprocessing
image = "D:\\NKB\\cranepos_data\\frames4\\1000.png"

image = cv2.imread(image)
time.sleep(.1)

# sharpen with a kernel
# kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# image = cv2.filter2D(image, -1, kernel)

# sharpen with gaussian blur
# image_blurred = cv2.GaussianBlur(image,(15,15), cv2.BORDER_DEFAULT)
# image = cv2.addWeighted(image, 2.0, image_blurred, -1.0, 0)

# play with contrast-brightness
# contrast=0.5
# brightness = 50
# image = cv2.addWeighted(image, contrast, image, 0, brightness)

# gamma transformation
# gamma = 1.95
# image = np.array(255*(image / 255) ** gamma, dtype = 'uint8')

# crop and resize
image = image[350:434, 500:650]


output = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize = (20,20))

plt.imshow(output)
plt.show()

In [ ]:
# # super resolution
# sr = cv2.dnn_superres.DnnSuperResImpl_create()
 
# path = "EDSR_x4.pb"

# sr.readModel(path)
 
# sr.setModel("edsr",4)
# result = sr.upsample(image)


In [ ]:
sr = cv2.dnn_superres.DnnSuperResImpl_create()
 
path = "random/FSRCNN_x3.pb"

sr.readModel(path)
 
sr.setModel("fsrcnn",3)
result = sr.upsample(image)

In [ ]:
# sr = cv2.dnn_superres.DnnSuperResImpl_create()
 
# path = "ESPCN_x3.pb"
 
# sr.readModel(path)
 
# sr.setModel("espcn",3)
 
# result = sr.upsample(image)

In [ ]:
# sharpen with a kernel
# kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
# result = cv2.filter2D(result, -1, kernel)

# sharpen with gaussian blur
# image_blurred = cv2.GaussianBlur(result,(15,15), cv2.BORDER_DEFAULT)
# result = cv2.addWeighted(result, 3.0, image_blurred, -2.0, 0)

# gamma transformation
gamma = 1.95
result = np.array(255*(result / 255) ** gamma, dtype = 'uint8')

# play with contrast-brightness
contrast=1.5
brightness = 0
result = cv2.addWeighted(result, contrast, result, 0, brightness)

In [ ]:
output = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
plt.figure(figsize = (20,20))

plt.imshow(output)
plt.show()

In [ ]:
image = result

In [ ]:

frame, pose, size = estimator_front(image, True)

output = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

output = display_pose(output, pose[:-1,[3]])

plt.imshow(output)
plt.show()
print(pose)

In [ ]:
estimator = PoseSingle(
    aruco_dict_type,
    1,
    n_markers,
    marker_poses,
    edge_len,
    k_video,
    d,
    apply_kf = False,
    transition_coef = 50,
    observation_coef = 0.02,
    x_bias = 1.25,
    # left_edge_weight_func  = f_left_x_04,
)

In [ ]:
# Test image preprocessing
image = "D:\\NKB\\cranepos_data\\frames5\\2700.png"

image = cv2.imread(image)
time.sleep(.1)

# sharpen with a kernel
kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
image = cv2.filter2D(image, -1, kernel)

# sharpen with gaussian blur
# image_blurred = cv2.GaussianBlur(image,(15,15), cv2.BORDER_DEFAULT)
# image = cv2.addWeighted(image, 2.0, image_blurred, -1.0, 0)

# play with contrast-brightness
contrast=1.5
brightness = -50
image = cv2.addWeighted(image, contrast, image, 0, brightness)

# gamma transformation
# gamma = 1.95
# image = np.array(255*(image / 255) ** gamma, dtype = 'uint8')

# crop and resize
# image = image[350:434, 500:650]

# apply super-resolution
# image = sr.upsample(image)

# OR(!) reesize image
image = cv2.resize(image,dsize=None,fx=3,fy=3)

# apply median blur
# image = cv2.medianBlur(image, 15)

output = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.figure(figsize = (20,20))

plt.imshow(output)
plt.show()

In [ ]:

frame, pose, size = estimator(image, True)

output = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

output = display_pose(output, pose[:-1,[3]])

plt.figure(figsize = (20,20))

plt.imshow(output)
plt.show()
print(pose)